In [1]:
import pandas as pd
import nltk
import re
import numpy as np
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences


basic_stopwords = stopwords.words('english')

df = pd.read_csv("dataset2.csv",encoding='latin')
df = df.drop(df.columns[[1,2,3,4]],axis=1)
df = df.rename(columns={df.columns[0]: 'target', df.columns[1]: 'tweet'})



def preprocess(tweet):
    tweet = tweet.lower() 
    tweet = re.sub('@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+',' ', tweet)
    tweet = nltk.word_tokenize(tweet)
    tweet = [word for word in tweet if word not in basic_stopwords and word.isalnum()]
    return tweet

df.tweet =df.tweet.apply(lambda x:preprocess(x)) 





model = Word2Vec(df.tweet,window=4,workers=4,min_count=1)
all_normed_vectors = model.wv.get_normed_vectors()
model.save('model.bin')



word_vectors = {}

file = open('model.txt',encoding='utf-8')

for line in file:
    vector_values = line.split()
    vector = np.asarray(vector_values[1:])
    word_vectors[vector_values[0]] = vector
    
    

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')

tokenizer.fit_on_texts(df.tweet)

sequences = tokenizer.texts_to_sequences(df.tweet)
padded = pad_sequences(sequences)


vocabulary = tokenizer.word_index

encoder = LabelEncoder()
encoder.fit_transform(df.target.to_list())

x_train = padded[:1200000]
x_test = padded[1200001:]
y_train = df.loc[1:1200000,'target'].values
y_test = df.loc[1200001:,'target'].values




num_words = len(vocabulary) + 1

embedded_matrix = np.zeros((num_words,100))
for word,i in vocabulary.items():
    embedded_vector = word_vectors.get(word)
    embedded_matrix[i] = embedded_vector

In [6]:
import numpy as np
# load embedding as a dict
def load_embedding(filename):
    # load embedding into memory, skip first line
    file = open(filename,'r',encoding="utf-8")
    lines = file.readlines()
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in lines:
        parts = line.split()
        # key is string word, value is numpy array for vector
        embedding[parts[0]] = np.asarray(parts[1:], dtype='float32')
    return embedding

# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, embedding_dim))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        vector = embedding.get(word)
        if vector is not None:
            weight_matrix[i] = vector
    return weight_matrix

# contains the index for each word
vocab = tokenizer.word_index
# total number of words in our vocabulary, plus one for unknown words
vocab_size = len(tokenizer.word_index) + 1
# embedding dimensions
embedding_dim = 200

# load embedding from file
raw_embedding = load_embedding('glove.twitter.27B.200d.txt')
# get vectors in the right order
embedding_matrix = get_weight_matrix(raw_embedding, vocab)



In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np
from numpy import asarray
from numpy import zeros
import nltk
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Embedding, LSTM
from keras.layers.core import Dense, Dropout
from keras.callbacks import ReduceLROnPlateau

embedding_layer = Embedding(num_words, 
                            200, 
                            weights = [embedding_matrix], 
                            input_length = 50, 
                            trainable = False)
# define model
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.2))
model.add(LSTM(200, dropout = 0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation = "sigmoid"))

print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 200)           67100600  
_________________________________________________________________
dropout (Dropout)            (None, 50, 200)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 200)               320800    
_________________________________________________________________
dense (Dense)                (None, 64)                12864     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 67,434,329
Trainable params: 333,729
Non-trainable params: 67,100,600
_________________________________________________________________
None


In [ ]:
BATCH_SIZE = 1024
EPOCHS = 15

model.compile(optimizer = "adam", loss = 'binary_crossentropy', metrics = ['accuracy'])

# callbacks
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', 
                              factor = 0.1,
                              min_lr = 0.01)

# train model
history = model.fit(x_train, y_train, batch_size = BATCH_SIZE, epochs = EPOCHS,
                    validation_split = 0.1, verbose = 1, callbacks = [reduce_lr])

Epoch 1/15
